# Machine Learning

<img src=ml_class.png>

Basicamente, *machine learning*- ou  aprendizado de máquina - é o estudo que permite automatizar uma tarefa sem a intervenção direta de um humano, usando modelos para extrair as informações de bases de dados.

No jargão, o modelo "treina" com base nos dados de entrada, e depois "prevê" resultados a medida que encontra novas situações.

Os modelos podem ser:

- **Supervisionados**: se o treino é realizado com a disponibilidade de variáveis respostas da base de treino
- **Não-supervisionados**: se o próprio algortimo é capaz de extrair as relações relevantes e inferir a variável resposta. Geralmente utilizado em clusterização

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.ion()
%matplotlib inline

## Modelos e algoritmos

Agora começamos algumas definições úteis

- **Modelo**: entendemos aqui como a formulação matemática que relaciona as variáveis de entrada com a(s) variável(is) de saída.
- **Algoritmo**: o processo matemático e estatítico usado para adequar - *fit* - o modelo aos dados.
- **Métrica de sucesso**: índice que será usado na avaliação da qualidade do resultado. Depende do modelo utilizado
- **Preprocessamento, ou *data prep***: preparação da base de dados para uso no algoritmo (selecionar/extrair variáveis, tratar valores faltantes, etc)

<img src=modelo.png width=500>

## *Overfitting* e *Underfitting*

Dois fenômenos, totalmente opostos e igualmente desastrosos, podem ocorrer aos se realizar o *fit* do modelo.

- ***Overfitting***: quando o modelo acaba por se adequar *demais* dos dados de treino, como se o algortimo tivesse decorado que tal entrada tem tal saída. Resultados podem estar longe da realidade para pontos não utilizados no *fit*. Dizemos que o modelo tem alta **variância**.
- ***Underfitting***: pelo contrário, quando o modelo quase não se adequa aos dados de treino. Dizemos que tem alto **viés**, pois apesar das informações apresentadas, pouco concede

<img src=over_under.png>

### Validação Cruzada

Uma maneira de contornar os problemas de *over* e *underfitting* é, em primeiro lugar, separar os dados em dois tipos: para treino e para teste. Assim temos maior controle quanto a qualidade do modelo nesse sentido.

#### *Overfitting*:

- **Característica**: Boa performance no *set* de treino, baixa no de teste.
- **Causa**: muitas variáveis, modelo muito complexo
- **Solução**: reduzir número de variáveis, regularização\*, mais pontos

#### *Underfitting*

- **Característica**: Baixa performance tanto no *set* de treino quanto no de teste
- **Causa**: modelo simples demais, muito rígido (regularização\*)
- **Solução**: mais variáveis

In [2]:
from sklearn.datasets import load_boston

<img src=boston_data.png>

In [3]:
x = load_boston()
df = pd.DataFrame(x.data, columns = x.feature_names)
df["MEDV"] = x.target
dfx = df.drop("MEDV",1) 
dfy = df["MEDV"]        
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
X = np.array(dfx)
y = np.array(dfy)

### Padronização

Para evitar que variáveis com maior escala dominem a função custo, dado que uma variação nelas influenciariam mais o resultado, é necessário padronizar a escala de todos os *inputs* numéricos.

Isso pode ser feito subtraindo o valor mínimo e dividindo pelo intervalo de variação:

$$ x_p = \frac{x - x_{mín}}{x_{máx} - x_{mín}}$$

Ou subtraindo o valor médio e dividindo pela variância:

$$ x_n = \frac{x - \bar{x}}{s_x}$$

In [5]:
from sklearn.preprocessing import StandardScaler



### *Feature Selection*

É importante, então, decidir quais informações valem de fato a pena serem usadas no modelo

Algumas formas principais de selecionar as variáveis:

#### Filtro

Seleciona variáveis com base na relação com a variável resposta. Independe do modelo

<img src='filter.png'>

#### Wrapper

Utiliza o próprio modelo para determinar a importância das variáveis.

<img src='wrapper.png'>

#### Embedded

Utiliza, além do modelo em si, a sua performance

<img src='embedded.png'>



## Regressão Linear
<br>
<br>
<img src=lin_reg.png width=400>

### Formulação

Na regressão linear, representamos a relação entre as variáveis independentes e depentendes no seguinte modelo:

$$y = \theta_0 + \theta_1x_1 + \theta_2x_2 + ... + \theta_nx_n + \epsilon$$

Ou, de forma matricial

$$ y = \Theta^{\it{T}}X + \epsilon$$

Onde $\Theta^{\it{T}} = [\theta_0\hspace{1.5mm}\theta_1 ... \theta_n]\hspace{1mm}$ e $\hspace{1mm}X = [1\hspace{1.5mm}x_1 ... x_n]^{\it{T}}$, e $\epsilon$ é o erro, que admitimos ter distribuição normal de média 0.

Nota-se que o termo **linear** se refere à relação entre os coeficientes. Sendo assim, o modelo $ y = a_1x_1 +a_2x_1^2+ a_3\log{x_1}$ é linear, pois basta substituir $x_2=x_1^2\hspace{1mm}$ e $\hspace{1mm}x_3=\log{x_1}$

#### Algoritmos

Como definimos anteriormente, já temos o modelo, agora falta definir como encontrar os coeficientes que mais adequam esse modelo aos nossos dados. Como estamos falando de **modelos supervisionados**, basicamente cada algoritmo tentará minimizar uma **função custo**, que mensiona o quanto os pontos estimados $\hat{y}$ estão divergentes dos pontos reais $y$.

##### Mínimos Quadrados

O mais simples dos algortimos, tem como função de custo a soma do quadrado das diferenças entre real e estimado, ou seja:

$$ FC = \sum_{j=0}^{M} (\hat{y}_j-y_j)^2 $$

Prefere-se, assim, vários pequenos erros a alguns erros grandes. 

Como consequência, o algoritmo se torna relativamente sensível a *outliers*, pontos que por acaso de distanciam do comportamento normal, já que o custo cresce com o quadrado da distância.



##### Ridge - Regularização $L_2$

A regularização tem por objetivo limitar a exploração do algoritmo quanto ao espaço de soluções possíveis. No caso do que chamamos de regularização $L_2$, adicionamos o quadrado dos coeficientes à função custo:

$$ FC = \sum_{j=0}^{M} (\hat{y}_j-y_j)^2 + \alpha\sum_{i=0}^{N} |\theta_i|^2 $$

O novo termo enrigece a solução, pois aumentos nos coeficientes serão penalizados, o que aumenta o viés, podendo ser utilizao para remediar o problema de *overfitting*.

Porém, mais um parâmetro deve ser definido: o valor de $\alpha$ que traz o melhor comprometimento viés-variância para a qualidade do modelo - maior $\alpha$ implica maior viés, sendo $\alpha=0$ o equivalente aos Mínimos Quadrados. Uma solução é a criação de uma terceira partição nos dados, além de treino e teste.

Com a primeira achamos o melhor $\Theta$ dado $\alpha$, a segunda usamos para definir o $\alpha$ mais adequado, e a terceira nos dá a medida da qualidade do modelo em si. 



##### Lasso - Regularização $L_1$

De forma semelhante ao **Ridge**, utiliza uma penalidade para os valores dos coeficientes, mas com a norma $l_1$. Sendo assim, este algoritmo costuma achar soluções em que os coeficientes das variáveis menos importantes são zerados. É comum usá-lo para selecionar variáveis.

$$ FC = \sum_{j=0}^{M} (\hat{y}_j-y_j)^2 + \alpha\sum_{i=0}^{N} |\theta_i| $$

Mesmas observações sobre o novo parâmetro $\alpha$ no algoritmo anterior.
<br>
<img src='L1_L2.png' width=500>

In [6]:
## Filtro



In [7]:
## Wrapper



In [8]:
## Embedded



### Métricas

Para avaliar a qualidade do modelo, temos a nosso dispor algumas métricas. No caso de regressões, as mais comuns são:

#### Variância explicada

Como o nome diz, nos dá o quanto o modelo consegue explicar da variação que observamos nos dados:

$$ VE(y,\hat{y}) = 1-\frac{Var(y-\hat{y})}{Var(y)}$$

#### Erro médio quadrado

Média no quadrado da diferença entre real e estimado

$$ EMQ = \frac{\sum_{j=1}^{M} (y-\hat{y})^2}{M+1}$$

#### Coeficiente de determinação - $R^2$

Nos trás o quão bem amostras futuras devem ser estimadas

$$ R^2=1-\frac{\sum_{j=0}^{M} (y-\hat{y})^2}{\sum_{j=0}^{M} (y-\bar{y})^2}$$

Notamos que $R^2$ é númericamente igual à variância explicada.

#### R ajustado - $R_{adj}^2$ ou $\bar{R^2}$

O coeficiente de determinação $R^2$ tende a aumentar com a adição de variáveis, independentemente de sua relevância para o modelo. Sendo assim, ao se comparar soluções com número diferente de *inputs*, devemos usar o R ajustado, que tenta controlar esse efeito (M: número de pontos; N: número de variáveis): 

$$ \bar{R^2}=1-(1-R^2)\frac{M-1}{M-N-1}$$




In [9]:
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
import numpy as np

def r2_aj (y_true, y_pred, shape):
    R2 = r2_score(y_true, y_pred)
    ra = 1-(1-R2)*(shape[0]-1)/(shape[0]-shape[1]-1)
    return ra
    

In [10]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

ln = LinearRegression()
rd = Ridge(alpha=0.5)
ls = Lasso(alpha=0.5)

In [11]:
## Regressão Linear




In [12]:
## Ridge




In [13]:
## Lasso




### Validação

Durante a decisão de escolha do modleo, é preciso ter controle da qualidade do que está sendo gerado. Se dependermos somente dos dados com que o algoritmo está treinando, corremos risco de *overfitting*, com muito erro ao se extrapolar para dados além do treino.

A solução mais simples é simular dados desconhecidos ao modelo, separando parte da base de treino, a ser usada somente na sua validação.


#### Hold-out

Método mais simples, dividem-se os dados históricos em *dataset* de treino e de validação. Apesar de se medir a qualidade do *fit* nos dados de treino, as decisões são tomadas com base na validação.

<img src=holdout.png>

#### Validação cruzada

Uma desvantagem do *hold-out* é a limitação que traz em termos de quantidade de pontos em que o modelo será treinado e validado. Se aumentarmos os pontos de treino, teremos menos pontos de validação, e vice-versa.

Como alternativa, podemos repetir a divisão treino-teste até passar por toda base de dados, efetuando a validação a cada passo. A performance geral do modelo é obtida da média da performance em cada divisão.

<img src=kfold.png>

#### Bootstrap (re-amostragem)

Com o *bootstrap*, um novo *dataset* é criado a partir de amostragens aleatórias - com reposição - dos dados que já temos.

Os dados históricos são entendidos como uma amostra da distribuição real dos dados, e o processo de *bootstrapping* procura recriar esse efeito.

<img src=bootstrap.png>


In [14]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample